# Tutorial 17 - Simulation

Thus far in this course we have focused our analysis efforts from data that comes directly from financial markets.  It is *real* data.

However, similar techniques can be applied to data that is simulated. *Simulation* is the process by which random number generators in a computer are  are used to create *fake* data.  The idea is that the simulated data resembles some aspect of the real-world, so the conclusions you draw from the fake data are applicable to real life.

Why might we want to simulate data in finance?  Here are a few of reaons:

1. The phenomenon you want to model is so complex that it's hard to derive pen and paper solutions. (This is what we are going to do in our example today).

2. There isn't enough historical data for you to feel comfortable about your conclusion, so you run simulations to generate more data.  (E.g. Risk management.)

3. Real world data, by definition, represents the past, and you believe the future is going to be different.  Thus, you run simulations to generate data that you feel reflects the future world better.

The purpose of this tutorial is to give a brief introduction to the style of coding that is involved when you run simulations in Python.  Towards this end, we will price a European put option using a technique called *Monte Carlo Simulation*.

### Load Packages

Let's begin by loading the packages that we will need.

In [205]:
##> import numpy as np



### Random Number Generation

At the heart of simulation is the generation of random numbers.  The `numpy.random` sublibrary has a variety of random number generators that we can use.

The following code generates 3 random numbers uniformly distributed between 0 and 1.

In [1]:
##> np.random.rand(3)



**Code Challenge:** What do you think the mean of a bunch of (0, 1) uniformly distributed numbers should be?  Test your hypothesis. 

The following code generates 5 standard normal random numbers.

In [2]:
##> np.random.randn(3)



There are a multitude of distributions available in `numpy.random`.  They are all listed in the `numpy` offical [documentation](https://docs.scipy.org/doc/numpy/reference/routines.random.html). 

### Setting the Random Seed with `numpy.random.seed()`

If you are typing along, you've notice that you are getting different random numbers.  This is because random number generators are really deterministic alogrithms that spit out sequences of numbers bases on a starting point, which is usually the internal system time of your computer.

We can also set the seed explicitly with the `random.seed()` method in the `numpy` package.

If you type the following code, you will get the same result.

In [3]:
##> np.random.seed(0)
##> np.random.randn(3)



**Note:** Don't set the seed in your production code.

### Geometric Brownian Motion

The series of trade prices for a stock is often modeled as a series of random variables, which is also referred to as a *stochastic process*. There many types of stochastic processes, and some of them resemble actual stock price movements better than others.

The Black-Scholes option pricing framework assumes that the price process of the underlying asset follows a *geometric brownian motion* (GBM). This means that:

1. The price process is continuous.

1. The log return over any period of time is normally distributed.

2. The returns during any two disjoint periods are independent.

While these assumptions are often violated in actual prices, the Black-Scholes framework lead to the first analytic solution to option pricing.  Moreover, GBMs are one of the simplest types of processes that reasonably model asset price dynamics, so it's often a good place to start when learning about simulating stock price data.

The price process of a geometric brownian motion is determined by the current risk-free rate $r$ and the annualized volatility of the underlying $\sigma$.  The prices that are separated by $\Delta t$ units of time are related by

$$S_{t} =  S_{t - \Delta t} \cdot \exp\bigg(\bigg(r - \frac{1}{2}\sigma^2\bigg)\Delta t + \sigma \sqrt{\Delta t} z_{t}\bigg)$$

where $z_{t}$ is a standard normal random variable.

This is called the Euler discretization of a GBM. It will serve as the  recipe for our simulation algorithm.  Note that the expression in the parentheses is simply the return factor between time $t - \Delta t$ and $t$.

### Option Pricing Objective

The option price we are going to price has the following parameters:

- underlying: QQQ
- current date: 11/16/2018
- expiration: 12/21/2018
- type: put
- strike: 160
- upx: 168
- d2x: 24

### Risk Neutral Pricing and Monte-Carlo

Consider our option pricing problem.  On November 16th, the underlying price at on expiration date, December 21, is a random variable.  The payoff of the put we are pricing is also a random variable, because it is simply a function of the expiration day underlying price.

One of the key theoretical insights of option pricing theory is that the price of an option is the (risk-neutral) *expected value* of the option payoff.  Said differently, the option price is the average option payoff, given the random nature of the underlying.

In mathematics, an expected value is defined as an integral.  Simple integrals can be solved with pen-and-paper.  In fact, the Black-Scholes option pricing formula is the solution to the expected value integral, given that the underlying proces is a geometric brownian motion.

Many integrals are too complicated to be solved with pen and paper.  However, there is another way to tackle these:  simulate the underlying distribution with a computer and take the average.  This is called Monte Carlo simulation.

This is not unlike the approach we took to solve the **Code Challenge** above.

### Pricing Procedure

So here is the approach we are going to take:

1. Simulate a bunch of QQQ prices paths, according to what we think the distribution of QQQ will be.
2. For each path, we will calculate the simulated payoff of our option on that path.
2. Our option price is goint to be the average of those simulated option payoffs.

### Market Calibration

We are simulating QQQ with a GBM.  Recall that a GBM process is completely determined by the risk-free rate $r$ an the volatility $\sigma$.  Here are the quantites we are going to use and why:

$r = 0$ - rates are low right now, and for short dated options the interest has little impact on the price of an option.

$\sigma = 0.2632$ - the implied vol for QQQ on that November 16th.

This process of looking at the current markets to inform your simulation is called *market calibration*.

### Simulating a Single Path

Our eventual pricing calculation is going to involve many hundreds of thousands of paths.  But let's first simulate one path, and calculate the option payoff for that path.

In particular, let's generate a path of daily prices, which means our price path is going to have a total of 25 prices, including the starting price.

In [221]:
##> # setting the random seed
##> np.random.seed(0)
##> 
##> # parameters of simulation
##> r = 0
##> sigma = 0.2632
##> dt = 1./252
##> 
##> # initializing paths
##> S = np.zeros(25)
##> S[0] = 167.50
##> 
##> # looping through days and generating steps in the paths
##> for t in range(1, 25):
##>     z = np.random.standard_normal(1)
##>     S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt
##>                             + sigma * math.sqrt(dt) * z)










Let's take a look at the path we generated:

In [4]:
##> S



Finally, we calulate the pay off for our put option as follows:

In [5]:
##> K = 160 # strike price
##> payoff = np.maximum(K - S[-1], 0)
##> payoff





### Simulating All Paths and Averaging

The Monte-Carlo simulation is simply repeating the above process a multitude of times, and then averaging all the simulated option payoffs.

The following code sets the parameters of our Monte Carlo pricer.  Notice that we are generating 1 million scenarios of each consisting of 24 steps.  That's a lot of calcuation.  In order to get reasonable performance we will exploit vectorized operations in `numpy`.

In [6]:
##> S0 = 168.
##> K = 160.
##> T = 24./252.
##> r = 0
##> sigma = 0.2632
##> M = 24
##> dt = T / M
##> I = 1000000




In the following code, `S` is being initialized to hold all the price paths. Notice that this is a matrix with 25 rows and 1 million columns.  Each column is going to be one price path.

In [7]:
##> S = np.zeros((M + 1, I))



Next, let's set the first price in each path as the starting underlying price.

In [8]:
##> S[0] = S0



Now, we'll fill in each path using the Euler discretization above.  There are 25 steps in the `for` loop.  At each step, 1 million normal random numbers are generated, and a vectorize operation is used to calculate the next price in all the paths simulaneously.  This is how we are able to exploit the performance of `numpy`.

In [9]:
##> for t in range(1, M + 1):
##>     z = np.random.standard_normal(I) 
##>     S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt
##>                             + sigma * math.sqrt(dt) * z)




**Code Challenge:** Extract the 967th path and print it to the screen.

As mentioned above, the price of our option is the average of the simulated option payoffs.

In [10]:
##> put_price = math.exp(-r * T) * np.mean(np.maximum(K - S[-1], 0))
##> round(put_price, 3)




### Further Reading

*P4F* - pp 59-68 (Monte Carlo Simulation)

*P4F* - pp 266-271 (Random Number Generation)